In [546]:
import pandas as pd
%matplotlib inline
%run Pipeline//upload_and_vizualize 
%run Pipeline//classify_and_evaluate 
%run Pipeline//aux
%run Pipeline//ULAB_ML_Pipeline
%run Pipeline//processing
%run Pipeline//cleaning
%run processing
import numpy as np 
import matplotlib.pyplot as plt
import pylab
import sys
import random
import sklearn as sk 
import json 
import re
from multiprocessing import Pool
from functools import partial
from time import time
from sklearn import svm, ensemble
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.linear_model import OrthogonalMatchingPursuit, RandomizedLogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.cross_validation import train_test_split, KFold
from sklearn.preprocessing import *
from sklearn.feature_selection import RFE
from sklearn.grid_search import ParameterGrid
from sklearn.svm import LinearSVC
from sklearn.metrics import *
import csv
from errno import EEXIST
from os import makedirs,path
from datetime import datetime as dr
from datetime import date
import numpy as np
import matplotlib
matplotlib.style.use('ggplot')
import pylab as pl
from upload_and_vizualize import camel_to_snake
from datetime import datetime as dt
from datetime import date

In [366]:
#### FAILED LABEL #####

In [538]:
def change_to_zero(series_row):
    if type(series_row) != str:
        return 0
    else:
        return 1

def change_to_zero_float(series_row):
    if series_row >= 1:
        return 1
    else:
        return 0
    
def generate_label(start_year, end_year):
    
    violhist = read_file('data/ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv')
    fce = read_file('data/ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv')

    
    date_types = ['year']
    date_format = df_dict['violation']['date_format']
    date_col = [df_dict['violation']['date_col']]
    #print(date_format,date_col)
    get_occupied_frame(violhist,date_col,date_format,date_types)

    date_col = [df_dict['inspection']['date_col']]
    get_occupied_frame(fce,date_col,date_format,date_types)

    fce = inspection[['PGM_SYS_ID','STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE','PROGRAM_CODES','ACTUAL_END_DATE','ACTUAL_END_DATE_year']]
    violhist = violhist[['PGM_SYS_ID','AGENCY_TYPE_DESC','AIR_LCON_CODE','ENF_RESPONSE_POLICY_CODE','POLLUTANT_CODES','PROGRAM_CODES','HPV_DAYZERO_DATE','HPV_DAYZERO_DATE_year']]
    violhist = violhist[violhist.ENF_RESPONSE_POLICY_CODE == 'HPV']
    #removing FRVs
    #violhist = violhist[violhist.ENF_RESPONSE_POLICY_CODE != 'FRV']
    violhist = violhist[violhist['HPV_DAYZERO_DATE_year'] >= start_year]
    violhist = violhist[violhist['HPV_DAYZERO_DATE_year'] <= end_year]
    
    #for fce
    fce = fce[fce['ACTUAL_END_DATE_year'] >= start_year]
    fce = fce[fce['ACTUAL_END_DATE_year'] <= end_year]
    
    merged_hpv_fce = pd.merge(violhist, fce, how='right', left_on=['PGM_SYS_ID', 'HPV_DAYZERO_DATE'], right_on=['PGM_SYS_ID','ACTUAL_END_DATE'])
    
    merged_hpv_fce['Outcome'] = merged_hpv_fce.ENF_RESPONSE_POLICY_CODE.apply(change_to_zero)
    #finding 0's
    
    output = merged_hpv_fce.groupby(['PGM_SYS_ID', 'ACTUAL_END_DATE_year']).sum().reset_index()
    
    output.Outcome = output.Outcome.apply(change_to_zero_float)
    
    return output, merged_hpv_fce

In [539]:
start_date = '2007'
end_date = '2016'
out, merged = generate_label(start_date, end_date)

In [540]:
out_dropped = out.dropna()
out_dropped.shape

(339994, 3)

In [542]:
merged.Outcome.value_counts()

0    687140
Name: Outcome, dtype: int64

In [435]:
## CONFIG DATA ##
START_DATE= '2007/01/01'
END_DATE = '2016/12/31'
fac_id = 'PGM_SYS_ID'
df_dict ={'violation': {#'data_file': 'data/ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv',
                        'interest_var': ['AGENCY_TYPE_DESC','AIR_LCON_CODE','ENF_RESPONSE_POLICY_CODE','POLLUTANT_CODES','PROGRAM_CODES','HPV_RESOLVED_DATE'],
                         'date_col': 'HPV_DAYZERO_DATE',
                       'date_format':'%m-%d-%Y'},
          
           'inspection': {'interest_var': ['STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE','PROGRAM_CODES'],
                         'date_col': 'ACTUAL_END_DATE',
                         'date_format':'%m-%d-%Y'},
          
           'stacktest': {'interest_var':['COMP_MONITOR_TYPE_CODE','POLLUTANT_CODES','AIR_STACK_TEST_STATUS_CODE'],
                        'date_col': 'ACTUAL_END_DATE',
                        'date_format':'%m/%d/%Y'},
          
           'titlev':{'interest_var':['COMP_MONITOR_TYPE_CODE','FACILITY_RPT_DEVIATION_FLAG'],
                        'date_col': 'ACTUAL_END_DATE',
                    'date_format':'%m/%d/%Y'},
          
           'formalact':{'interest_var':['ENF_TYPE_CODE','PENALTY_AMOUNT'],
                        'date_col': 'SETTLEMENT_ENTERED_DATE',
                       'date_format':'%m/%d/%Y'},
          
           'informalact':{'interest_var':['ENF_TYPE_CODE'],
                        'date_col': 'ACHIEVED_DATE',
                         'date_format':'%m/%d/%Y'}}

#general_df = pd.DataFrame()

In [494]:
def generate_features(start_date, end_date):
    #violation, inspection, titlev, stacktest, formalact, informalact = general_read_file(df_dict, START_DATE, END_DATE)
    
    violation_df = process_violation(violation, start_date, end_date)
    inspection_df = process_titlev(titlev, start_date, end_date)
    stacktest_df = process_stacktest(stacktest, start_date, end_date)
    formalact_df = process_formalact(formalact, start_date, end_date)
    informalact_df = process_informalact(informalact, start_date, end_date)
    noninspectHPV_df = process_noninspectHPV(violation, inspection, start_date, end_date)
    
    final_inspect_viol_df = pd.merge(inspection_df, violation_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_w_iv_stacktest_df = pd.merge(final_inspect_viol_df.dropna(), stacktest_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_w_ivs_formal_df = pd.merge(final_w_iv_stacktest_df, formalact_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_w_ivsf_informal_df = pd.merge(final_w_ivs_formal_df, informalact_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_w_ivsfi_noninspect_df = pd.merge(final_w_ivsf_informal_df, noninspectHPV_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])

    re_separate = r'(.[^_]*)_(.*)'
    sep = lambda x: pd.Series([i for i in re.split(re_separate,x)])
    final_id_year = final_w_ivsfi_noninspect_df['id_+_date'].apply(sep)
    final_df =pd.concat([final_id_year.rename(columns={1:'PGM_SYS_ID',2:'HPV_DAYZERO_DATE_year'}), final_w_ivsfi_noninspect_df], axis=1)
    final_df.drop([0, 3, 'id_+_date'], axis = 1, inplace = True)

    return final_inspect_viol_df, final_w_iv_stacktest_df, final_w_ivs_formal_df, final_w_ivsf_informal_df, final_w_ivsfi_noninspect_df,final_df

In [437]:
## READ INITIAL FILTERED FILE ##
violation, inspection, titlev, stacktest, formalact, informalact = general_read_file(df_dict, START_DATE, END_DATE)

In [413]:
FACILITY = 'data/ICIS-AIR_downloads/ICIS-AIR_FACILITIES.csv'
VIOLATION = 'data/ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv'
INSPECTION = 'data/ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv'
FORMALACT = 'data/ICIS-AIR_downloads/ICIS-AIR_FORMAL_ACTIONS.csv'
INFORMALACT = 'data/ICIS-AIR_downloads/ICIS-AIR_INFORMAL_ACTIONS.csv'
STACKTEST = 'data/ICIS-AIR_downloads/ICIS-AIR_STACK_TESTS.csv'
TITLEV = 'data/ICIS-AIR_downloads/ICIS-AIR_TITLEV_CERTS.csv'

#facilities = read_file('ICIS-AIR_downloads/ICIS-AIR_FACILITIES.csv')
#pd_violations = read_file('ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv')
#pd_inspection = read_file('ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv')
#pd_formalact = read_file('ICIS-AIR_downloads/ICIS-AIR_FORMAL_ACTIONS.csv')
#pd_informalact = read_file('ICIS-AIR_downloads/ICIS-AIR_INFORMAL_ACTIONS.csv')
#pd_stacktest = read_file('ICIS-AIR_downloads/ICIS-AIR_STACK_TESTS.csv')
#pd_titlev = read_file('ICIS-AIR_downloads/ICIS-AIR_TITLEV_CERTS.csv')

In [495]:
#Checking for feature generation
start_date = '2007'
end_date = '2016'
final_inspect_viol_df, final_w_iv_stacktest_df, final_w_ivs_formal_df, final_w_ivsf_informal_df, final_w_ivsfi_noninspect_df, feat = generate_features(start_date, end_date)

(9436, 237)

In [517]:
feat = feat.dropna()

In [508]:
pd.set_option('max_columns',1000)
feat.head()

,PGM_SYS_ID,HPV_DAYZERO_DATE_year,COMP_MONITOR_TYPE_CODE_TVA,FACILITY_RPT_DEVIATION_FLAG,AGENCY_TYPE_DESC_Local,AGENCY_TYPE_DESC_State,AGENCY_TYPE_DESC_U.S. EPA,AIR_LCON_CODE_ACH,AIR_LCON_CODE_AVA,AIR_LCON_CODE_BAA,AIR_LCON_CODE_BUN,AIR_LCON_CODE_CCH,AIR_LCON_CODE_CHC,AIR_LCON_CODE_COA,AIR_LCON_CODE_CUS,AIR_LCON_CODE_EDC,AIR_LCON_CODE_FCO,AIR_LCON_CODE_GBU,AIR_LCON_CODE_HUN,AIR_LCON_CODE_IMP,AIR_LCON_CODE_JEF,AIR_LCON_CODE_JFC,AIR_LCON_CODE_KCA,AIR_LCON_CODE_KNX,AIR_LCON_CODE_LIN,AIR_LCON_CODE_LRA,AIR_LCON_CODE_MBU,AIR_LCON_CODE_MCA,AIR_LCON_CODE_MDA,AIR_LCON_CODE_MEC,AIR_LCON_CODE_NCA,AIR_LCON_CODE_NCU,AIR_LCON_CODE_NDC,AIR_LCON_CODE_NSI,AIR_LCON_CODE_NSO,AIR_LCON_CODE_None,AIR_LCON_CODE_ORC,AIR_LCON_CODE_PAM,AIR_LCON_CODE_PCA,AIR_LCON_CODE_PLK,AIR_LCON_CODE_PMC,AIR_LCON_CODE_PNC,AIR_LCON_CODE_PSC,AIR_LCON_CODE_SAC,AIR_LCON_CODE_SBC,AIR_LCON_CODE_SCA,AIR_LCON_CODE_SDS,AIR_LCON_CODE_SHA,AIR_LCON_CODE_SHL,AIR_LCON_CODE_SJV,AIR_LCON_CODE_SLO,AIR_LCON_CODE_SPC,AIR_LCON_CODE_SWC,AIR_LCON_CODE_TUC,AIR_LCON_CODE_VCA,AIR_LCON_CODE_WCD,AIR_LCON_CODE_YRC,AIR_LCON_CODE_YSA,POLLUTANT_CODES_10193,POLLUTANT_CODES_10214,POLLUTANT_CODES_10243,POLLUTANT_CODES_10305,POLLUTANT_CODES_10358,POLLUTANT_CODES_10361,POLLUTANT_CODES_10366,POLLUTANT_CODES_10373,POLLUTANT_CODES_10461,POLLUTANT_CODES_10486,POLLUTANT_CODES_10494,POLLUTANT_CODES_12289,POLLUTANT_CODES_200000481,POLLUTANT_CODES_2049,POLLUTANT_CODES_2448,POLLUTANT_CODES_300000002,POLLUTANT_CODES_300000003,POLLUTANT_CODES_300000004,POLLUTANT_CODES_300000005,POLLUTANT_CODES_300000006,POLLUTANT_CODES_300000010,POLLUTANT_CODES_300000011,POLLUTANT_CODES_300000013,POLLUTANT_CODES_300000015,POLLUTANT_CODES_300000016,POLLUTANT_CODES_300000017,POLLUTANT_CODES_300000018,POLLUTANT_CODES_300000020,POLLUTANT_CODES_300000025,POLLUTANT_CODES_300000028,POLLUTANT_CODES_300000035,POLLUTANT_CODES_300000036,POLLUTANT_CODES_300000037,POLLUTANT_CODES_300000038,POLLUTANT_CODES_300000039,POLLUTANT_CODES_300000042,POLLUTANT_CODES_300000046,POLLUTANT_CODES_300000047,POLLUTANT_CODES_300000048,POLLUTANT_CODES_300000068,POLLUTANT_CODES_300000069,POLLUTANT_CODES_300000072,POLLUTANT_CODES_300000074,POLLUTANT_CODES_300000076,POLLUTANT_CODES_300000079,POLLUTANT_CODES_300000082,POLLUTANT_CODES_300000090,POLLUTANT_CODES_300000094,POLLUTANT_CODES_300000095,POLLUTANT_CODES_300000097,POLLUTANT_CODES_300000099,POLLUTANT_CODES_300000107,POLLUTANT_CODES_300000119,POLLUTANT_CODES_300000120,POLLUTANT_CODES_300000129,POLLUTANT_CODES_300000138,POLLUTANT_CODES_300000145,POLLUTANT_CODES_300000147,POLLUTANT_CODES_300000149,POLLUTANT_CODES_300000151,POLLUTANT_CODES_300000152,POLLUTANT_CODES_300000153,POLLUTANT_CODES_300000168,POLLUTANT_CODES_300000169,POLLUTANT_CODES_300000171,POLLUTANT_CODES_300000173,POLLUTANT_CODES_300000174,POLLUTANT_CODES_300000177,POLLUTANT_CODES_300000178,POLLUTANT_CODES_300000179,POLLUTANT_CODES_300000180,POLLUTANT_CODES_300000182,POLLUTANT_CODES_300000183,POLLUTANT_CODES_300000184,POLLUTANT_CODES_300000188,POLLUTANT_CODES_300000189,POLLUTANT_CODES_300000207,POLLUTANT_CODES_300000208,POLLUTANT_CODES_300000209,POLLUTANT_CODES_300000210,POLLUTANT_CODES_300000218,POLLUTANT_CODES_300000220,POLLUTANT_CODES_300000222,POLLUTANT_CODES_300000223,POLLUTANT_CODES_300000224,POLLUTANT_CODES_300000226,POLLUTANT_CODES_300000228,POLLUTANT_CODES_300000231,POLLUTANT_CODES_300000234,POLLUTANT_CODES_300000236,POLLUTANT_CODES_300000240,POLLUTANT_CODES_300000241,POLLUTANT_CODES_300000242,POLLUTANT_CODES_300000243,POLLUTANT_CODES_300000248,POLLUTANT_CODES_300000254,POLLUTANT_CODES_300000282,POLLUTANT_CODES_300000289,POLLUTANT_CODES_300000290,POLLUTANT_CODES_300000291,POLLUTANT_CODES_300000292,POLLUTANT_CODES_300000294,POLLUTANT_CODES_300000295,POLLUTANT_CODES_300000298,POLLUTANT_CODES_300000301,POLLUTANT_CODES_300000302,POLLUTANT_CODES_300000304,POLLUTANT_CODES_300000305,POLLUTANT_CODES_300000307,POLLUTANT_CODES_300000310,POLLUTANT_CODES_300000312,POLLUTANT_CODES_300000313,POLLUTANT_CODES_300000318,POLLUTANT_CODES_300000319,POLLUTANT_CODE

In [469]:
check_na(final_w_ivs_formal_df)

value,False,True
variable,,
AGENCY_TYPE_DESC_Local,6217,127391
AGENCY_TYPE_DESC_State,6217,127391
AGENCY_TYPE_DESC_U.S. EPA,6217,127391
AIR_LCON_CODE_ACH,6217,127391
AIR_LCON_CODE_AVA,6217,127391
AIR_LCON_CODE_BAA,6217,127391
AIR_LCON_CODE_BUN,6217,127391
AIR_LCON_CODE_CCH,6217,127391
AIR_LCON_CODE_CHC,6217,127391


In [470]:
check_na(final_w_ivsf_informal_df)

value,False,True
variable,,
AGENCY_TYPE_DESC_Local,6217,127391
AGENCY_TYPE_DESC_State,6217,127391
AGENCY_TYPE_DESC_U.S. EPA,6217,127391
AIR_LCON_CODE_ACH,6217,127391
AIR_LCON_CODE_AVA,6217,127391
AIR_LCON_CODE_BAA,6217,127391
AIR_LCON_CODE_BUN,6217,127391
AIR_LCON_CODE_CCH,6217,127391
AIR_LCON_CODE_CHC,6217,127391


In [473]:
check_na(merged)

value,False,True
variable,,
ACTUAL_END_DATE,687863,0
ACTUAL_END_DATE_datetime,687863,0
ACTUAL_END_DATE_year,687863,0
AGENCY_TYPE_DESC,4784,683079
AIR_LCON_CODE,1973,685890
COMP_MONITOR_TYPE_CODE,687863,0
ENF_RESPONSE_POLICY_CODE,4784,683079
HPV_DAYZERO_DATE,4784,683079
HPV_DAYZERO_DATE_datetime,4784,683079


In [472]:
check_na(final_w_ivsfi_noninspect_df)

value,False,True
variable,,
AGENCY_TYPE_DESC_Local,9436,127391
AGENCY_TYPE_DESC_State,9436,127391
AGENCY_TYPE_DESC_U.S. EPA,9436,127391
AIR_LCON_CODE_ACH,9436,127391
AIR_LCON_CODE_AVA,9436,127391
AIR_LCON_CODE_BAA,9436,127391
AIR_LCON_CODE_BUN,9436,127391
AIR_LCON_CODE_CCH,9436,127391
AIR_LCON_CODE_CHC,9436,127391


In [491]:
check_na(final_w_ivfi_noninspect_df)

value,False,True
variable,,
AGENCY_TYPE_DESC_Local,9436,0
AGENCY_TYPE_DESC_State,9436,0
AGENCY_TYPE_DESC_U.S. EPA,9436,0
AIR_LCON_CODE_ACH,9436,0
AIR_LCON_CODE_AVA,9436,0
AIR_LCON_CODE_BAA,9436,0
AIR_LCON_CODE_BUN,9436,0
AIR_LCON_CODE_CCH,9436,0
AIR_LCON_CODE_CHC,9436,0


In [487]:
check_na(final_inspect_viol_df)

value,False,True
variable,,
AGENCY_TYPE_DESC_Local,6217,127391
AGENCY_TYPE_DESC_State,6217,127391
AGENCY_TYPE_DESC_U.S. EPA,6217,127391
AIR_LCON_CODE_ACH,6217,127391
AIR_LCON_CODE_AVA,6217,127391
AIR_LCON_CODE_BAA,6217,127391
AIR_LCON_CODE_BUN,6217,127391
AIR_LCON_CODE_CCH,6217,127391
AIR_LCON_CODE_CHC,6217,127391


In [529]:
y = pd.merge(feat, merged_dropped, how='inner', left_on=['PGM_SYS_ID','HPV_DAYZERO_DATE_year'], right_on = ['PGM_SYS_ID','ACTUAL_END_DATE_year'])

In [530]:
feat.shape

(3119, 237)

In [531]:

merged_dropped.shape

(1938, 16)

In [532]:
y.shape

(9785, 252)

In [534]:
y.Outcome.value_counts(

1    9785
Name: Outcome, dtype: int64